In [47]:
import numpy as np
import pandas as pd 
from pandas import Series,DataFrame
import pandas_profiling
from datetime import datetime
import xgboost as xgb 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge

In [48]:
# read the data 
train_data = pd.read_csv('train.csv',skipinitialspace=True,parse_dates=['purchase_date','release_date'])
#train_data1 = train_data.loc[train_data['playtime_forever']!=0]
test_data = pd.read_csv('test.csv',skipinitialspace=True,parse_dates=['purchase_date','release_date'])
unit_data = pd.concat([train_data,test_data],axis=0) 
unit_data.reset_index(drop=True,inplace=True)
unit_data.to_csv('union_data.csv')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [49]:
#data preprocess
positive_reviews_mean = unit_data['total_positive_reviews'].dropna().mean()
unit_data['total_positive_reviews'] = unit_data['total_positive_reviews'].fillna(positive_reviews_mean)
negative_reviews_mean = unit_data['total_negative_reviews'].dropna().mean()
unit_data['total_negative_reviews'] = unit_data['total_negative_reviews'].fillna(negative_reviews_mean)

In [50]:
#del the unknown data
unit_data['purchase_date'] = unit_data['purchase_date'].fillna(unit_data['release_date'])
updated_train_data = unit_data.drop(index=76) 
data_copy = updated_train_data

In [51]:
#classify the price
bins=[-1,5000,10000,15000,20000,1000000000] 
data_copy['tprice'] = pd.cut(data_copy['price'],bins,labels=['a','b','c','d','e'])


In [52]:
#the days bettween the purchase_data and today
temp = pd.DatetimeIndex(data_copy['purchase_date'])
temp1=data_copy['purchase_date'].apply(lambda x:datetime.today()-x)
days = temp1.apply(lambda x:x.days)
#data_copy['days'] = days 

In [53]:
#scale the data
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
positive_param = scaler.fit(data_copy['total_positive_reviews'].values.reshape(-1,1))
data_copy['positive_scaled'] = scaler.fit_transform(data_copy['total_positive_reviews'].values.reshape(-1,1), positive_param)
negative_param = scaler.fit(data_copy['total_negative_reviews'].values.reshape(-1,1))
data_copy['negative_scaled'] = scaler.fit_transform(data_copy['total_negative_reviews'].values.reshape(-1,1), negative_param)
days_param = scaler.fit(days.values.reshape(-1,1)) 
data_copy['days_scaled'] = scaler.fit_transform(days.values.reshape(-1,1),days_param)

In [54]:
#calculate the rate of the positive reviews
data_copy['rate'] = data_copy['total_positive_reviews']/(data_copy['total_positive_reviews']+data_copy['total_negative_reviews'])

In [55]:
#process the nan data
from math import isnan
rate_mean = data_copy['rate'].dropna().mean()
for i,v in data_copy['rate'].items():
    if isnan(v):
        data_copy['rate'][i] = rate_mean
data_copy['rate'][65]


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0.8490740500229329

In [56]:
#classify genres,is_free, tprice,categories,tags
dummies_genres = updated_train_data['genres'].str.get_dummies(sep=",").add_prefix('genres')
dummies_is_free = pd.get_dummies(data_copy['is_free'],prefix='is_free')
dummies_tprice = pd.get_dummies(data_copy['tprice'],prefix='tprice')
dummies_categories = updated_train_data['categories'].str.get_dummies(sep=",").add_prefix('categories') 
dummies_tags = updated_train_data['tags'].str.get_dummies(sep=",").add_prefix('tags') 


In [57]:
#concat the dataframe
data_copy = pd.concat([data_copy,dummies_genres,dummies_is_free,dummies_tprice,dummies_categories,dummies_tags],axis=1) 

In [58]:
data_copy.drop(['is_free','price','genres','categories','tags','purchase_date','release_date','total_positive_reviews','total_negative_reviews','tprice'],axis=1,inplace=True)

In [59]:
data_copy.to_csv('whole_data1.csv',index=False)

In [60]:
#divide the data into train set and test set
data = data_copy
x_train = data.ix[0:356, 2:]  
y_train = data.ix[0:356,1:2] 
x_test = data.ix[357:,2:] 
x_test

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


,positive_scaled,negative_scaled,days_scaled,rate,genresAction,genresAdventure,genresAnimation & Modeling,genresAudio Production,genresCasual,genresDesign & Illustration,...,tagsVoxel,tagsWalking Simulator,tagsWar,tagsWargame,tagsWarhammer 40K,tagsWestern,tagsWorld War I,tagsWorld War II,tagsZombies,tagseSports
357,-0.292716,-0.085301,-0.784604,0.699115,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
358,-0.215194,-0.035090,-1.700832,0.720520,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,-0.339893,-0.129919,-1.687003,0.837805,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360,-0.355127,-0.134160,0.152369,0.632075,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
361,0.634530,0.031363,1.327907,0.915827,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
362,-0.347264,-0.132265,-0.276357,0.826923,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
363,-0.316771,-0.119678,-1.559076,0.818913,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
364,-0.350483,-0.132175,-0.023962,0.755162,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
365,-0.313208,-0.077722,-0.307474,0.578844,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
366,-0.261633,-0.090535,1.718600,0.793768,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#train the model
from sklearn.linear_model import Ridge
rcv = Ridge()      
rcv.fit(x_train,y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [62]:
y_predict = rcv.predict(x_test)
y_predict

array([[ 20.97536057],
       [  4.09423661],
       [ -6.92777473],
       [  6.64434387],
       [ 36.0966485 ],
       [ -1.73669555],
       [ -1.66456907],
       [  1.57149658],
       [ 12.98690316],
       [ -1.45822471],
       [ 12.83905289],
       [  1.23395999],
       [ 14.39117129],
       [  5.44849527],
       [ 11.44157418],
       [ -9.74747371],
       [  7.71755134],
       [ -2.69659609],
       [ -1.3808063 ],
       [-18.32786693],
       [ 11.46957433],
       [ -0.08291309],
       [ -2.00218497],
       [  0.81548008],
       [ -1.23187608],
       [ -7.7668564 ],
       [ -0.27204968],
       [ -9.02524503],
       [ -4.6669493 ],
       [  0.34416282],
       [ 24.22148377],
       [ 17.10137933],
       [ -6.86764874],
       [ -3.74776284],
       [  0.32277949],
       [  2.88275497],
       [-13.4856484 ],
       [ 16.62236219],
       [  8.41889479],
       [  3.15553827],
       [  0.96102809],
       [ -6.25075634],
       [ 24.8921633 ],
       [  7

In [63]:
#change the negative data to the zero
[rows,cols] = y_predict.shape
for i in range(rows):
    for j in range(cols):
        if y_predict[i,j]< 0:
            y_predict[i,j] = 0
y_predict

array([[20.97536057],
       [ 4.09423661],
       [ 0.        ],
       [ 6.64434387],
       [36.0966485 ],
       [ 0.        ],
       [ 0.        ],
       [ 1.57149658],
       [12.98690316],
       [ 0.        ],
       [12.83905289],
       [ 1.23395999],
       [14.39117129],
       [ 5.44849527],
       [11.44157418],
       [ 0.        ],
       [ 7.71755134],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [11.46957433],
       [ 0.        ],
       [ 0.        ],
       [ 0.81548008],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.34416282],
       [24.22148377],
       [17.10137933],
       [ 0.        ],
       [ 0.        ],
       [ 0.32277949],
       [ 2.88275497],
       [ 0.        ],
       [16.62236219],
       [ 8.41889479],
       [ 3.15553827],
       [ 0.96102809],
       [ 0.        ],
       [24.8921633 ],
       [ 7.66625536],
       [ 0.        ],
       [ 0

In [65]:
#output the result
prediction = pd.DataFrame(y_predict,columns=['playtime_forever'])
prediction.reset_index(drop = True,inplace = True)
prediction['id'] = range(len(prediction))
order = ['id','playtime_forever']
prediction = prediction[order]
prediction.to_csv('submission.csv',index = False)